# Reading GEDI L4A with Xarray

This notebook explores if reading GEDI L4A data with Xarray can be faster than h5py.

Install Xarray...
```
conda install -c conda-forge mambda
# Then create a new env for Xarray
mamba create -n xarrayenv -c conda-forge xarray netCDF4 h5py ipykernel geopandas h5netcdf s3fs
conda activate xarrayenv
python3 -m ipykernel install --user
```

In [21]:
!unset PROJ_LIB

In [43]:
import xarray as xr
import h5py
import geopandas as gpd
import s3fs
import os

In [23]:
sample = '/projects/maap-documentation-examples/gedi-subset/data/GEDI04_A_2019108045816_O01962_01_T01066_02_002_01_V002.h5'
sample2 = '/projects/shared-buckets/alexdevseed/gedi-l4a/gabon/GEDI04_A_2019146134206_O02558_02_T05641_02_002_01_V002.h5'
sample2_https = 'https://maap-ops-workspace.s3.amazonaws.com/shared/alexdevseed/gedi-l4a/gabon/GEDI04_A_2019146134206_O02558_02_T05641_02_002_01_V002.h5'
sample2_s3 = 's3://maap-ops-workspace/shared/alexdevseed/gedi-l4a/gabon/GEDI04_A_2019146134206_O02558_02_T05641_02_002_01_V002.h5'
filter_cols = ["agbd", "agbd_se", "l4_quality_flag", "sensitivity", "lat_lowestmode", "lon_lowestmode"]

In [24]:
# h5py does not work with s3 by default, need to try s3ro
h5 = h5py.File(sample, 'r')
#h5_s3 = h5py.File(sample2_s3, driver ='ros3')


In [31]:
print(h5.keys())
h5['BEAM0000']['geolocation'].keys()

<KeysViewHDF5 ['ANCILLARY', 'BEAM0000', 'BEAM0001', 'BEAM0010', 'BEAM0011', 'BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011', 'METADATA']>


<KeysViewHDF5 ['elev_lowestmode_a1', 'elev_lowestmode_a10', 'elev_lowestmode_a2', 'elev_lowestmode_a3', 'elev_lowestmode_a4', 'elev_lowestmode_a5', 'elev_lowestmode_a6', 'lat_lowestmode_a1', 'lat_lowestmode_a10', 'lat_lowestmode_a2', 'lat_lowestmode_a3', 'lat_lowestmode_a4', 'lat_lowestmode_a5', 'lat_lowestmode_a6', 'lon_lowestmode_a1', 'lon_lowestmode_a10', 'lon_lowestmode_a2', 'lon_lowestmode_a3', 'lon_lowestmode_a4', 'lon_lowestmode_a5', 'lon_lowestmode_a6', 'sensitivity_a1', 'sensitivity_a10', 'sensitivity_a2', 'sensitivity_a3', 'sensitivity_a4', 'sensitivity_a5', 'sensitivity_a6', 'shot_number', 'stale_return_flag']>

In [40]:
# TODO switch to the fsspec method for reading from cloud buckets
s3 = s3fs.S3FileSystem(anon=False)
fileset = s3.open(sample2_s3)
with s3.open(sample2_s3, mode='rb') as s3f:
    with h5py.File(s3f, mode='r') as f:
        print(list(f.keys()))
        #x5 = xr.open_dataset(f, group = 'BEAM0000/geolocation', engine='scipy')
    #x5_s3 = xr.open_dataset(fileset, group = 'BEAM0000', engine='netcdf4')


x5 = xr.open_dataset(sample, group = 'BEAM0000')
# TODO what's the right way to use fsspec here?
#x5 = xr.open_dataset(fileset, group = 'BEAM0000', 
#                     engine='netcdf4')
x5
#print(x5_s3)

['ANCILLARY', 'BEAM0000', 'BEAM0001', 'BEAM0010', 'BEAM0011', 'BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011', 'METADATA']


<xarray.Dataset>
Dimensions:               (phony_dim_4: 63659, phony_dim_5: 4)
Coordinates:
    delta_time            (phony_dim_4) float64 ...
    lat_lowestmode        (phony_dim_4) float64 ...
    lon_lowestmode        (phony_dim_4) float64 ...
Dimensions without coordinates: phony_dim_4, phony_dim_5
Data variables: (12/26)
    agbd                  (phony_dim_4) float32 ...
    agbd_pi_lower         (phony_dim_4) float32 ...
    agbd_pi_upper         (phony_dim_4) float32 ...
    agbd_se               (phony_dim_4) float32 ...
    agbd_t                (phony_dim_4) float32 ...
    agbd_t_se             (phony_dim_4) float32 ...
    ...                    ...
    selected_mode_flag    (phony_dim_4) uint8 ...
    sensitivity           (phony_dim_4) float32 ...
    shot_number           (phony_dim_4) uint64 ...
    solar_elevation       (phony_dim_4) float32 ...
    surface_flag          (phony_dim_4) uint8 ...
    xvar                  (phony_dim_4, phony_dim_5) float32 ...
Attributes:
    description:  Coverage beam

In [28]:
%time
df = x5.drop([item for item in list(x5.data_vars) if item not in filter_cols]).to_dataframe().reset_index()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [29]:
%time
gdf = gpd.GeoDataFrame(df.loc[:,~df.columns.isin(['lon_lowestmode', 'lat_lowestmode', 'delta_time', 'phony_dim_4'])], 
                       geometry=gpd.points_from_xy(df.lon_lowestmode, df.lat_lowestmode))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


In [45]:
def get_geo_boundary(*, iso: str, level: int) -> gpd.GeoDataFrame:
    file_path = f'/projects/my-public-bucket/iso3/{iso}-ADM{level}.json'
    
    if not os.path.exists(file_path):
        r = requests.get(
            'https://www.geoboundaries.org/gbRequest.html',
            dict(ISO=iso, ADM=f'ADM{level}')
        )
        r.raise_for_status()
        dl_url = r.json()[0]['gjDownloadURL']
        geo_boundary = requests.get(dl_url).json()

        with open(file_path, 'w') as out:
            out.write(json.dumps(geo_boundary))
    
    return gpd.read_file(file_path)


aoi = get_geo_boundary(iso='GAB', level=0)
aoi

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,Gabon,GAB,GAB-ADM0-3_0_0-B1,GAB,ADM0,"MULTIPOLYGON (((8.83154 -0.92271, 8.83809 -0.9..."


In [50]:
%time
# Spatial Filter
gdf.crs = "EPSG:4326"

# TODO: is it faster with a spatial index, or rough pass with BBOX first?
bbox = aoi.geometry[0].bounds

gdf_clip = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
gdf_gsrm = gdf_clip[gdf_clip['geometry'].within(aoi.geometry[0])]
gdf_gsrm.shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


(8022, 5)

In [51]:
%time
#gdf.shape
#gdf.plot()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


(63659, 5)

In [ ]:
# try s3 path with xarray
# find nested sensitivity_a2
# spatial filter with a polygon ?
# all beams with mf_dataset ?
# subgroups
# xarray s3 paths with fsspec